In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import lxml

In [2]:
# IMPORT CSV of WATCHLIST - must save with UTF-8 encoding in Excel

watchlistDF = pd.read_csv('import/WATCHLIST.csv')
watchlistDF = watchlistDF.drop(['Position', 'Created','Modified','Description'], axis=1)
watchlistDF = watchlistDF[['Title','Year','Runtime (mins)','Directors','Genres','Const','URL','IMDb Rating','Num Votes','Your Rating', 'Date Rated']]
watchlistDF.head()

,Title,Year,Runtime (mins),Directors,Genres,Const,URL,IMDb Rating,Num Votes,Your Rating,Date Rated
0,Avengers: Endgame,2019,181,"Anthony Russo, Joe Russo","Action, Adventure, Sci-Fi",tt4154796,https://www.imdb.com/title/tt4154796/,8.6,551638,NaN,NaN
1,The Godfather,1972,175,Francis Ford Coppola,"Crime, Drama",tt0068646,https://www.imdb.com/title/tt0068646/,9.2,1464382,NaN,NaN
2,The Godfather: Part II,1974,202,Francis Ford Coppola,"Crime, Drama",tt0071562,https://www.imdb.com/title/tt0071562/,9.0,1018758,NaN,NaN
3,The Dark Knight,2008,152,Christopher Nolan,"Action, Crime, Drama, Thriller",tt0468569,https://www.imdb.com/title/tt0468569/,9.0,2097524,NaN,NaN
4,Pulp Fiction,1994,154,Quentin Tarantino,"Crime, Drama",tt0110912,https://www.imdb.com/title/tt0110912/,8.9,1673510,NaN,NaN


In [3]:
# ============= Search Result Scraping =============
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,&num_votes=10000,&genres=comedy&lists=%21watchlist&count=250'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object
soup = BeautifulSoup(response.text)

# Get titles, urls, const from results
titleList = []
fullUrlList = []
constList = []

results = soup.find_all('h3', class_='lister-item-header')

for result in results:
    titleText = result.find('a').text
    titleList.append(titleText)
    
    #grab shortened URL
    aTag = result.find('a')
    url = aTag['href']
    
    #turn into full URL & add to list
    fullUrl = 'https://www.imdb.com' + url
    fullUrlList.append(fullUrl)

    #strip out const & add to list
    const = url.replace('/title/','')
    const = const.replace('/','')
    constList.append(const)


In [4]:
# Finding const
fullUrlList = []
constList = []

results = soup.find_all('h3', class_='lister-item-header')

for result in results:
    #grab shortened URL
    aTag = result.find('a')
    url = aTag['href']
    
    #turn into full URL & add to list
    fullUrl = 'https://www.imdb.com' + url
    fullUrlList.append(fullUrl)

    #strip out const & add to list
    const = url.replace('/title/','')
    const = const.replace('/','')
    constList.append(const)

In [5]:
# Get years from results
yearList = []

results = soup.find_all('span', class_='lister-item-year text-muted unbold')

for result in results:
    year = result.text
    year = year[-5:]
    year = year.replace(')', '')
    yearList.append(year)

In [6]:
ratingList = []
runtimeList = []
genreList = []

# Get Rating
ratingResults = soup.find_all('span', class_='certificate')
for result in ratingResults:
    ratingList.append(result.text)

# Get Runtime
rumtimeResults = soup.find_all('span', class_='runtime')
for result in rumtimeResults:
    runtimeList.append(result.text)

# Get Genre
genreResults = soup.find_all('span', class_='genre')
for result in genreResults:
    genreList.append(result.text)

In [7]:
# Get Rating
ratingResults = soup.find_all('div', class_='inline-block ratings-metascore')

metaScoreList = []

for rating in ratingResults:
    metaRating = rating.find('span').text
    metaRating = metaRating.replace('        ','')
    metaScoreList.append(metaRating)

In [8]:
# Build Dataframe from Search
fullDF = pd.DataFrame(list(zip(titleList, yearList, runtimeList, ratingList, genreList, metaScoreList, constList, fullUrlList)),
                 columns = ['Title', 'Year', 'Runtime', 'Rating', 'Genres', 'MetaScore', 'Const', 'URL'])

# Format year and score as numbers
fullDF.loc[fullDF['Year'].notnull(), 'Year'] = fullDF.loc[fullDF['Year'].notnull(), 'Year'].apply(int)
fullDF.loc[fullDF['MetaScore'].notnull(), 'MetaScore'] = fullDF.loc[fullDF['MetaScore'].notnull(), 'MetaScore'].apply(int)

searchResultDF = fullDF
fullDF.head()

,Title,Year,Runtime,Rating,Genres,MetaScore,Const,URL
0,Hustlers,2019,110 min,R,"\nComedy, Crime, Drama",79,tt5503686,https://www.imdb.com/title/tt5503686/
1,Once Upon a Time... in Hollywood,2019,161 min,R,"\nComedy, Drama",83,tt7131622,https://www.imdb.com/title/tt7131622/
2,Toy Story 4,2019,100 min,G,"\nAnimation, Adventure, Comedy",84,tt1979376,https://www.imdb.com/title/tt1979376/
3,Yesterday,2019,116 min,PG-13,"\nComedy, Fantasy, Music",56,tt8079248,https://www.imdb.com/title/tt8079248/
4,Parasite,2019,132 min,R,"\nComedy, Drama, Thriller",95,tt6751668,https://www.imdb.com/title/tt6751668/


In [9]:
watchlistDF.head()

,Title,Year,Runtime (mins),Directors,Genres,Const,URL,IMDb Rating,Num Votes,Your Rating,Date Rated
0,Avengers: Endgame,2019,181,"Anthony Russo, Joe Russo","Action, Adventure, Sci-Fi",tt4154796,https://www.imdb.com/title/tt4154796/,8.6,551638,NaN,NaN
1,The Godfather,1972,175,Francis Ford Coppola,"Crime, Drama",tt0068646,https://www.imdb.com/title/tt0068646/,9.2,1464382,NaN,NaN
2,The Godfather: Part II,1974,202,Francis Ford Coppola,"Crime, Drama",tt0071562,https://www.imdb.com/title/tt0071562/,9.0,1018758,NaN,NaN
3,The Dark Knight,2008,152,Christopher Nolan,"Action, Crime, Drama, Thriller",tt0468569,https://www.imdb.com/title/tt0468569/,9.0,2097524,NaN,NaN
4,Pulp Fiction,1994,154,Quentin Tarantino,"Crime, Drama",tt0110912,https://www.imdb.com/title/tt0110912/,8.9,1673510,NaN,NaN


In [10]:
# Compare watchlist & search results ---- x = search, y = watchlist
mergedDF = searchResultDF.merge(watchlistDF,on='Const', how='outer')

# Drop all non Nan rows (movies in the search, not in watchlist)
mergedDF = mergedDF[pd.isnull(mergedDF['Title_y'])]
mergedDF

# Create new DF for movies to watch
moviesToWatch = mergedDF
moviesToWatch = moviesToWatch[['Title_x', 'Year_x', 'Runtime', 'Rating', 'Genres_x', 'MetaScore', 'Const', 'URL_x']]
moviesToWatch = moviesToWatch.rename(columns={'Title_x':'Title', 'Year_x':'Year', 'Genres_x':'Genres', 'URL_x':'URL'})

# Remove \n in Genres
moviesToWatch = moviesToWatch.replace('\nComedy','Comedy')

moviesToWatch.head()


,Title,Year,Runtime,Rating,Genres,MetaScore,Const,URL
0,Hustlers,2019,110 min,R,"\nComedy, Crime, Drama",79,tt5503686,https://www.imdb.com/title/tt5503686/
1,Once Upon a Time... in Hollywood,2019,161 min,R,"\nComedy, Drama",83,tt7131622,https://www.imdb.com/title/tt7131622/
2,Toy Story 4,2019,100 min,G,"\nAnimation, Adventure, Comedy",84,tt1979376,https://www.imdb.com/title/tt1979376/
3,Yesterday,2019,116 min,PG-13,"\nComedy, Fantasy, Music",56,tt8079248,https://www.imdb.com/title/tt8079248/
4,Parasite,2019,132 min,R,"\nComedy, Drama, Thriller",95,tt6751668,https://www.imdb.com/title/tt6751668/


In [11]:
# Write Both DFs (Search & Watchlist) to XLSX
writer = pd.ExcelWriter('export/imdb_list_v2.xlsx', engine='xlsxwriter')

searchResultDF.to_excel(writer, sheet_name='Search Results', index=False)
watchlistDF.to_excel(writer, sheet_name='Watchlist', index=False)
moviesToWatch.to_excel(writer, sheet_name='Movies To Watch', index=False)

writer.save()